# Database Questions

*This worksheet was originally designed by [Erin George](https://www.math.ucla.edu/~egeo/) (Department of Mathematics, UCLA). It has been subsequently revised by later TAs and instructors.*

Today, we will be working with SQLite3 and Pandas to gain some practice working with large datasets.

Of course, the first thing we need to do is import our modules:

In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt # optional, in case we want to plot

The dataset we will be working with is the US Wildfires dataset available on [kaggle](https://www.kaggle.com/rtatman/188-million-us-wildfires).  Downloading it from there requires a kaggle account.  You can either create a kaggle account or you can go on CCLE, where I've uploaded it.  Either way, put the database (.sqlite file) in the same directory as this worksheet, then run the following code block:

In [2]:
conn = sqlite3.connect('FPA_FOD_20170508.sqlite')

A brief summary of the dataset: the data contains information on US wildfires between the years 1992 to 2015.  Each fire has information associated with it, such as how big it is, where it started, when it started, and so on.

This database contains A LOT of information:

In [3]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('spatial_ref_sys',), ('spatialite_history',), ('sqlite_sequence',), ('geometry_columns',), ('spatial_ref_sys_aux',), ('views_geometry_columns',), ('virts_geometry_columns',), ('geometry_columns_statistics',), ('views_geometry_columns_statistics',), ('virts_geometry_columns_statistics',), ('geometry_columns_field_infos',), ('views_geometry_columns_field_infos',), ('virts_geometry_columns_field_infos',), ('geometry_columns_time',), ('geometry_columns_auth',), ('views_geometry_columns_auth',), ('virts_geometry_columns_auth',), ('sql_statements_log',), ('SpatialIndex',), ('ElementaryGeometries',), ('KNN',), ('Fires',), ('idx_Fires_Shape',), ('idx_Fires_Shape_node',), ('idx_Fires_Shape_rowid',), ('idx_Fires_Shape_parent',), ('NWCG_UnitIDActive_20170109',)]


Fortunately, almost all the information we need is in the 'Fires' table!

Unfortunately, that means we wouldn't get any practice using joins.  Let's modify our data so that we split some information between separate tables in a reasonable way.

The code below is new, but it is just the command to create new tables from SELECT commands and the command to drop a table.  Once you run the code below, comment it out.  This will modify the database file and generate an error if the modification has already been made.

In [4]:
q = cursor.execute("CREATE TABLE fire_sizes AS SELECT FOD_ID, FIRE_SIZE, FIRE_SIZE_CLASS FROM Fires;")
q = cursor.execute("CREATE TABLE fire_times AS SELECT FOD_ID, FIRE_YEAR, DISCOVERY_DATE, DISCOVERY_DOY, DISCOVERY_TIME, CONT_DATE, CONT_DOY, CONT_TIME FROM Fires;")
q = cursor.execute("CREATE TABLE fire_locs AS SELECT FOD_ID, LONGITUDE, LATITUDE, COUNTY, STATE FROM Fires;")
q = cursor.execute("CREATE TABLE fire_causes AS SELECT FOD_ID, STAT_CAUSE_CODE, STAT_CAUSE_DESCR FROM Fires;")
q = cursor.execute("DROP TABLE Fires;")

The tables now are `fire_sizes`, `fire_times`, `fire_locs` and `fire_causes`.  These tables contain the following information, organized by `FOD_ID`, which uniquely identifies each fire:
- `fire_size`: The size of the fire, in acres.  Also, the fire size class, which is a classification used to group up fires by size.
- `fire_times`: The date and time the fire was discovered and then controlled.  Also, the year the fire took place.
- `fire_locs`: The state, county, longitude, and latitude of the fire (where it started).
- `fire_causes`: The cause of each fire, as a code and a description.

Let's get some intial information from this table.  For example, we could ask the question "how many fires in this dataset were from California?"

In [5]:
q = cursor.execute("SELECT FOD_ID FROM fire_locs WHERE STATE='CA'")
result = cursor.fetchall() # get all results
len(result)

189550

Alternatively, we could ask a question such as "how many fires in this dataset happened since the year 2000?"

In [6]:
q = cursor.execute("SELECT FOD_ID FROM fire_times WHERE FIRE_YEAR>=2000")
result = cursor.fetchall()
len(result)

1308317

Now that we've split our data into multiple tables, some questions require information in two tables.  For example, "how many fires in this dataset happened in California since 2000?"

In [7]:
q = cursor.execute(\
    """SELECT T.FOD_ID
       FROM fire_times T
       LEFT JOIN fire_locs L ON T.FOD_ID = L.FOD_ID 
       WHERE T.FIRE_YEAR>=2000 AND L.STATE='CA'""")
result = cursor.fetchall()
len(result)

121535

We can actually do the above in SQL itself with a clever choice of an accumulation function (COUNT):

In [8]:
q = cursor.execute(\
    """SELECT COUNT(T.FOD_ID)
       FROM fire_times T
       LEFT JOIN fire_locs L ON T.FOD_ID = L.FOD_ID 
       WHERE T.FIRE_YEAR>=2000 AND L.STATE='CA'
       GROUP BY L.STATE""")
result = cursor.fetchall()
result

[(121535,)]

## Group Questions

Let's ask a more complicated question.  Below is a list of three questions we can ask with this dataset.  You can decide which of the three questions you'd like to answer.  In discussion, you'll then form groups of 2-3 people to work together on that question.  These questions are intentionally open-ended, so write whatever code or do whatever data analysis you feel is necessary to answer the question.  Each question will have a few different subquestions to get you thinking of ideas: you can do any or all of them.  If your group finishes your question, go ahead and tackle another one!

Lastly, try to do the questions with as little non-SQL code as possible!  It's okay if you need more than just SQL, but you should be able to do at least one JOIN, GROUP BY, and accumulation in SQL before you do anything with Pandas to answer the subquestions.

**Question 1:** How has the intensity of the fires changed over time? (rooms 1-4)

*Sub-question 1a:* What is the average size of a fire each year?

In [9]:
cmd = \
    """SELECT T.FIRE_YEAR, AVG(S.FIRE_SIZE)
       FROM fire_times T
       LEFT JOIN fire_sizes S ON T.FOD_ID = S.FOD_ID 
       GROUP BY T.FIRE_YEAR"""
df = pd.read_sql_query(cmd, conn)

df

,FIRE_YEAR,AVG(S.FIRE_SIZE)
0,1992,32.364222
1,1993,35.356208
2,1994,54.198875
3,1995,28.676755
4,1996,79.460604
5,1997,52.313024
6,1998,29.414983
7,1999,68.039342
8,2000,79.234760
9,2001,43.009601


*Sub-question 1b:* How has the number of class G fires (the largest) changed over the years?

In [10]:
cmd = \
    """SELECT T.FIRE_YEAR, COUNT(T.FOD_ID)
       FROM fire_times T
       LEFT JOIN fire_sizes S ON T.FOD_ID = S.FOD_ID
       WHERE S.FIRE_SIZE_CLASS='G'
       GROUP BY T.FIRE_YEAR"""
df = pd.read_sql_query(cmd, conn)

df

,FIRE_YEAR,COUNT(T.FOD_ID)
0,1992,58
1,1993,65
2,1994,146
3,1995,69
4,1996,201
5,1997,61
6,1998,76
7,1999,155
8,2000,245
9,2001,113


*Sub-question 1c:* What is the proportion of fires that are class A (the smallest) each year?

In [11]:
cmd = \
    """SELECT T.FIRE_YEAR, COUNT(T.FOD_ID)
       FROM fire_times T
       LEFT JOIN fire_sizes S ON T.FOD_ID = S.FOD_ID
       WHERE S.FIRE_SIZE_CLASS='A'
       GROUP BY T.FIRE_YEAR"""
df_A = pd.read_sql_query(cmd, conn)
cmd = \
    """SELECT T.FIRE_YEAR, COUNT(T.FOD_ID)
       FROM fire_times T
       LEFT JOIN fire_sizes S ON T.FOD_ID = S.FOD_ID
       GROUP BY T.FIRE_YEAR"""
df_all = pd.read_sql_query(cmd, conn)
df = df_A.merge(df_all, on='FIRE_YEAR')
df['Ratio'] = df['COUNT(T.FOD_ID)_x'] / df['COUNT(T.FOD_ID)_y']
df.drop(columns=['COUNT(T.FOD_ID)_x', 'COUNT(T.FOD_ID)_y'])

df

,FIRE_YEAR,COUNT(T.FOD_ID)_x,COUNT(T.FOD_ID)_y,Ratio
0,1992,25540,67975,0.375726
1,1993,20767,61989,0.335011
2,1994,27900,75955,0.367323
3,1995,23913,71472,0.334579
4,1996,25120,75574,0.332389
5,1997,21664,61450,0.352547
6,1998,23408,68370,0.342372
7,1999,27562,89363,0.308427
8,2000,33063,96416,0.342920
9,2001,31131,86587,0.359534


**Question 2:** How has the location of the fires changed over time? (rooms 5-8)

*Sub-question 2a:* How has the average position of the fires changed over time?

In [12]:
cmd = \
    """SELECT T.FIRE_YEAR, AVG(L.LONGITUDE), AVG(L.LATITUDE)
       FROM fire_times T
       LEFT JOIN fire_locs L ON T.FOD_ID = L.FOD_ID 
       GROUP BY T.FIRE_YEAR"""
df = pd.read_sql_query(cmd, conn)

df

,FIRE_YEAR,AVG(L.LONGITUDE),AVG(L.LATITUDE)
0,1992,-98.719338,37.484116
1,1993,-96.399318,36.259484
2,1994,-98.925340,37.858960
3,1995,-95.350225,36.792188
4,1996,-97.491818,36.589964
5,1997,-95.751267,36.966139
6,1998,-95.388802,36.816101
7,1999,-94.508781,36.534526
8,2000,-94.246959,36.076239
9,2001,-94.011471,36.857829


*Sub-question 2b:* For each state, in what years did it have at least 4000 wildfires?

In [13]:
cmd = \
    """SELECT T.FIRE_YEAR, L.STATE, COUNT(T.FOD_ID)
       FROM fire_times T
       LEFT JOIN fire_locs L ON T.FOD_ID = L.FOD_ID 
       GROUP BY T.FIRE_YEAR, L.STATE"""
df = pd.read_sql_query(cmd, conn)
display(df[df['COUNT(T.FOD_ID)'] >= 4000].groupby('STATE')['FIRE_YEAR'].apply(list))

STATE
AL                             [1995, 1996, 1999, 2000]
AZ                                   [1993, 1994, 1995]
CA    [1992, 1993, 1994, 1995, 1996, 1997, 1998, 199...
FL    [1992, 1993, 1996, 1998, 1999, 2000, 2001, 200...
GA    [1992, 1993, 1994, 1995, 1996, 1997, 1998, 199...
KS                                               [2015]
LA                                               [2000]
MS           [1992, 1993, 1995, 1996, 1999, 2000, 2006]
NC    [1992, 1993, 1994, 1995, 1996, 1997, 1998, 199...
NY    [2003, 2004, 2005, 2006, 2007, 2008, 2009, 201...
PR                                               [2005]
SC     [1992, 1993, 1994, 1996, 1999, 2000, 2001, 2002]
TX    [2005, 2006, 2007, 2008, 2009, 2010, 2011, 201...
Name: FIRE_YEAR, dtype: object

*Sub-question 2c:* What proportion of fires were in California each year?

In [14]:
cmd = \
    """SELECT T.FIRE_YEAR, COUNT(T.FOD_ID)
       FROM fire_times T
       LEFT JOIN fire_locs L ON T.FOD_ID = L.FOD_ID 
       WHERE L.STATE='CA'
       GROUP BY T.FIRE_YEAR, L.STATE"""
df_cal = pd.read_sql_query(cmd, conn)
cmd = \
    """SELECT T.FIRE_YEAR, COUNT(T.FOD_ID)
       FROM fire_times T
       LEFT JOIN fire_sizes S ON T.FOD_ID = S.FOD_ID
       GROUP BY T.FIRE_YEAR"""
df_all = pd.read_sql_query(cmd, conn)
df = df_cal.merge(df_all, on='FIRE_YEAR')
df['Ratio'] = df['COUNT(T.FOD_ID)_x'] / df['COUNT(T.FOD_ID)_y']
df = df.drop(columns=['COUNT(T.FOD_ID)_x', 'COUNT(T.FOD_ID)_y'])

df

,FIRE_YEAR,Ratio
0,1992,0.159367
1,1993,0.133411
2,1994,0.113936
3,1995,0.103271
4,1996,0.121378
5,1997,0.129048
6,1998,0.100395
7,1999,0.099706
8,2000,0.072364
9,2001,0.094495


**Question 3:** How do the causes of the fires vary by location? (rooms 9-12)

*Sub-question 3a:* What's the most common cause of fire in each state? (This is much easier if you are familiar with `df.pivot_table` or `df.pivot`, although it is doable without both of them.)

In [15]:
cmd = \
    """SELECT L.STATE, C.STAT_CAUSE_DESCR, COUNT(L.FOD_ID)
       FROM fire_locs L
       LEFT JOIN fire_causes C ON L.FOD_ID = C.FOD_ID 
       GROUP BY C.STAT_CAUSE_DESCR, L.STATE"""
df = pd.read_sql_query(cmd, conn)
df = df.pivot_table(index='STATE', columns='STAT_CAUSE_DESCR')
df = df.fillna(0)
df
df.idxmax(axis=1)

STATE
AK            (COUNT(L.FOD_ID), Lightning)
AL                (COUNT(L.FOD_ID), Arson)
AR                (COUNT(L.FOD_ID), Arson)
AZ            (COUNT(L.FOD_ID), Lightning)
CA        (COUNT(L.FOD_ID), Miscellaneous)
CO            (COUNT(L.FOD_ID), Lightning)
CT        (COUNT(L.FOD_ID), Miscellaneous)
DC        (COUNT(L.FOD_ID), Miscellaneous)
DE        (COUNT(L.FOD_ID), Miscellaneous)
FL            (COUNT(L.FOD_ID), Lightning)
GA       (COUNT(L.FOD_ID), Debris Burning)
HI    (COUNT(L.FOD_ID), Missing/Undefined)
IA    (COUNT(L.FOD_ID), Missing/Undefined)
ID            (COUNT(L.FOD_ID), Lightning)
IL        (COUNT(L.FOD_ID), Miscellaneous)
IN       (COUNT(L.FOD_ID), Debris Burning)
KS        (COUNT(L.FOD_ID), Miscellaneous)
KY       (COUNT(L.FOD_ID), Debris Burning)
LA        (COUNT(L.FOD_ID), Miscellaneous)
MA        (COUNT(L.FOD_ID), Miscellaneous)
MD       (COUNT(L.FOD_ID), Debris Burning)
ME    (COUNT(L.FOD_ID), Missing/Undefined)
MI       (COUNT(L.FOD_ID), Debris Burning)
MN   

*Sub-question 3b:* Which states have more fires from lightning than arson?

In [16]:
cmd = \
    """SELECT L.STATE, COUNT(L.FOD_ID)
       FROM fire_locs L
       LEFT JOIN fire_causes C ON L.FOD_ID = C.FOD_ID
       WHERE C.STAT_CAUSE_DESCR='Arson'
       GROUP BY L.STATE"""
arson = pd.read_sql_query(cmd, conn)
cmd = \
    """SELECT L.STATE, COUNT(L.FOD_ID)
       FROM fire_locs L
       LEFT JOIN fire_causes C ON L.FOD_ID = C.FOD_ID
       WHERE C.STAT_CAUSE_DESCR='Lightning'
       GROUP BY L.STATE"""
lightning = pd.read_sql_query(cmd, conn)
df = arson.merge(lightning, on='STATE', how='outer')
df = df.fillna(0)
df['STATE'][df['COUNT(L.FOD_ID)_y'] > df['COUNT(L.FOD_ID)_x']]

0     AK
3     AZ
4     CA
5     CO
9     FL
13    ID
26    MT
29    NE
32    NM
33    NV
37    OR
42    SD
45    UT
48    WA
51    WY
Name: STATE, dtype: object

*Sub-question 3c:* What is the proportion of fires in each state that is caused by children? (These are labeled 'Children' in `STAT_CAUSE_DESCR`)

In [17]:
cmd = \
    """SELECT L.STATE, COUNT(L.FOD_ID)
       FROM fire_locs L
       LEFT JOIN fire_causes C ON L.FOD_ID = C.FOD_ID
       WHERE C.STAT_CAUSE_DESCR='Children'
       GROUP BY L.STATE"""
child = pd.read_sql_query(cmd, conn)
cmd = \
    """SELECT L.STATE, COUNT(L.FOD_ID)
       FROM fire_locs L
       LEFT JOIN fire_causes C ON L.FOD_ID = C.FOD_ID
       GROUP BY L.STATE"""
total = pd.read_sql_query(cmd, conn)
df = child.merge(total, on='STATE', how='outer')
df = df.fillna(0)
df['Ratio'] = df['COUNT(L.FOD_ID)_x']/df['COUNT(L.FOD_ID)_y']
df.drop(columns=['COUNT(L.FOD_ID)_x','COUNT(L.FOD_ID)_y'])

,STATE,Ratio
0,AK,0.044304
1,AL,0.015352
2,AR,0.011970
3,AZ,0.035635
4,CA,0.036560
5,CO,0.006090
6,CT,0.057677
7,DC,0.030303
8,DE,0.005848
9,FL,0.048903
